In [2]:
import urllib
import warnings
from pathlib import Path as p

from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
#from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI

warnings.filterwarnings("ignore")

In [3]:
data_folder = p.cwd() / "data"
p(data_folder).mkdir(parents=True, exist_ok=True)

In [6]:
#Reading saving accounts related pdfs from bankofireland.com website

main_url="https://personalbanking.bankofireland.com/app/uploads/"

docs=["Customer-Info-Sheet-SuperSaver-Account-1.pdf",\
      "Customer-Infomation-Sheet-Advantage-Fixed-Term-Deposit.pdf",\
      "2570642-GoalSaver-Customer-Information-Sheet-V2-final.pdf"]

In [108]:
def retrieve_docs(main_url,docs):
    """
    Retrieves documents from specified URLs and extracts their first page content.

    Parameters:
    - main_url (str): The base URL to which the document paths will be appended.
    - docs (list of str): A list of document paths to be appended to the main URL for retrieval.

    Steps:
    1. Iterates over the document paths in 'docs'.
    2. Constructs the full URL for each document and retrieves it as a PDF.
    3. Loads each PDF and extracts the content of the first page.
    4. Stores the first page content of each document in a list.
    5. Concatenates the contents of all first pages into a single string.

    Returns:
    tuple: A tuple containing two elements:
        - pages (list): A list of objects representing the first page of each retrieved document.
        - context (str): A string representing the concatenated contents of the first pages of all documents.
    """
    
    pages=[]
    for doc in docs:
        pdf_url = main_url + doc
        pdf_file = str(p(data_folder, doc))

        urllib.request.urlretrieve(pdf_url, pdf_file)
        pdf_loader = PyPDFLoader(pdf_file)
        page = pdf_loader.load_and_split()
        pages.append(page[0])
        
    
    context = " ".join(str(p.page_content) for p in pages)
    
    return pages,context


In [109]:
pages,context=retrieve_docs(main_url,docs)

In [110]:
pages

[Document(page_content='Minimum monthly amount €5\nMaximum monthly amount €2,500\nAccess You have instant access to your money.\nAdditional lodgements No\nWithdrawal arrangementsYou can withdraw money in any Bank of Ireland branch (you will need to bring \nphoto ID with you), by 365 online or via phone banking when registered for \nthese services.\nInterest rate type Fixed – The interest rate stays the same for the 12 months.\nInterest paymentInterest is paid at the end of the 12 month fixed rate period. See our Customer \nDeposits Rate Sheet on bankofireland.com  for current rates.\nSole or joint accounts Sole and joint accounts permitted.\nHow to applyYou can open this account online through the Bank of Ireland website via the \nmobile app or in any Bank of Ireland branch.\nTo open the account online, visit the savings page on bankofireland.com  and \ngo to the SuperSaver product page. Click the button “Apply now”.\nOther important informationTo open an account, you must have a perso

In [111]:
def generate_answer(context, question):
    """
    Generates an answer to a given question based on the provided context using Vertex AI models.

    Parameters:
    - context (str): The context text which the answer should be based on.
    - question (str): The question for which the answer is sought.

    Steps:
    1. Initializes two Vertex AI models:
       - 'text-bison@001' for Large Language Model (LLM) text generation.
       - 'textembedding-gecko@001' for embeddings.
    2. Constructs a prompt template for answering questions using the provided context.
    3. Creates a QA chain using the 'text-bison@001' model with the specified prompt.
    4. Generates an answer using the QA chain based on the given context and question.

    Returns:
    str: The generated answer, or "answer not available in context" if the answer cannot be derived from the provided context.
    
    """   
    
    vertex_llm_text = VertexAI(model_name="text-bison@001")
    #vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001") 
    
       
    prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                        not contained in the context, say "answer not available in context" \n\n
                        Context: \n {context}?\n
                        Question: \n {question} \n
                        Answer:
                      """

    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context","question"]
    )
    stuff_chain = load_qa_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)
    stuff_answer = stuff_chain(
    {"input_documents": pages, "question": question}, return_only_outputs=True
    )
    return stuff_answer["output_text"]

In [116]:
question='What savings accounts are there?'

In [117]:
generate_answer(context,question)

'There are 3 savings accounts: SuperSaver Account, Advantage Fixed Term Deposit account and GoalSaver Account.'

In [118]:
question='How much can I save with different accounts?'

In [119]:
generate_answer(context,question)

'The minimum monthly amount for the SuperSaver account is €5 and the maximum monthly amount is €2,500. The minimum monthly amount for the Advantage Fixed Term Deposit account is €5,000 and the maximum monthly amount is not specified. The minimum monthly amount for the GoalSaver account is €20 and the maximum monthly amount is €2,000.'

In [120]:
question='Write a very short summary of all accounts including the information about amounts and rates'

In [121]:
generate_answer(context,question)

'The SuperSaver Account has a minimum monthly amount of €5 and a maximum monthly amount of €2,500. The interest rate is fixed for 12 months and is paid at the end of the 12 month fixed rate period. The Advantage Fixed Term Deposit account has a minimum lodgement of €5,000 and a maximum lodgement of €5,000. The interest rate is fixed for the term of the account and is paid at maturity of the account. The GoalSaver Account has a minimum monthly amount of €20 and a maximum monthly amount of €2,000.'

In [122]:
question='What can I save for?'

In [123]:
generate_answer(context,question)

'You can save for a short term holiday or new car, a long term wedding or house extension, or you can just save for a rainy day fund.'

In [124]:
question='Give me the rates'

In [125]:
generate_answer(context,question)

'The interest rate is variable and may increase or decrease at the discretion of Bank of Ireland.'

In [126]:
question='Give me the rates of all accounts'

In [127]:
generate_answer(context,question)

'The interest rate for the SuperSaver Account is 0.05%.\nThe interest rate for the Advantage Fixed Term Deposit Account is 0.05%.\nThe interest rate for the GoalSaver Account is 0.05%.'

In [128]:
question='I am broke. Is there a chance for me?'

In [129]:
generate_answer(context,question)

'Yes, you can open a GoalSaver account with a minimum monthly amount of €20.'

In [130]:
question='Which of these accounts would amaze me?'

In [131]:
generate_answer(context,question)

'The SuperSaver Account would amaze you.'

In [132]:
question='Why SuperSaver Account would amaze me?'

In [133]:
generate_answer(context,question)

'The SuperSaver Account offers a fixed rate for 12 months, along with a range of extra benefits.'

In [134]:
question='If you were me which one would you pick?'

In [135]:
generate_answer(context,question)

'I would pick the GoalSaver Account because it has a higher interest rate than the other two accounts.'

In [138]:
question='Does GoalSaver Account have a higher interest rate than the other two accounts'

In [139]:
generate_answer(context,question)

'The interest rate for the GoalSaver Account is variable, which means that it can change at any time. The interest rate for the Advantage Fixed Term Deposit Account is fixed, which means that it will not change for the term of the account. The interest rate for the SuperSaver Account is fixed for 12 months, which means that it will not change for the first 12 months of the account.'

In [144]:
question='What if I forget I have a savings account?'

In [145]:
generate_answer(context,question)

'You can contact the bank and they will be able to help you find your account.'

In [146]:
question='What do you personally think about these accounts?'

In [147]:
generate_answer(context,question)

'I think that these accounts are great for people who are looking to save money. They offer a variety of different options, so you can find the one that best suits your needs. I also like that they offer competitive interest rates, so you can earn money on your savings.'

In [148]:
question='What if there is a crisis?'

In [149]:
generate_answer(context,question)

'The interest rate may increase or decrease at the discretion of Bank of Ireland.'

In [150]:
question='What if I inherit a ton of money?'

In [91]:
generate_answer(context,question)

'You can open an Advantage Fixed Term Deposit account in any Bank of Ireland branch.'

In [151]:
question='What if I inherit some money?'

In [152]:
generate_answer(context,question) #it sees the difference between a ton of money and some money :)

'You can open a GoalSaver account and save for your goal.'

In [97]:
def summarize(context):
    
    vertex_llm_text = VertexAI(model_name="text-bison@001")
    vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001") 
    
       
    prompt_template = """Summarize the context in a few lines \n\n
                        Context: \n {context}?\n                       
                        Answer:
                      """

    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context"]
    )
    stuff_chain = load_qa_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)
    stuff_answer = stuff_chain(
    {"input_documents": pages}, return_only_outputs=True
    )
    return stuff_answer["output_text"]

In [98]:
summarize(context)

'The context is about 3 saving accounts offered by Bank of Ireland. The accounts are SuperSaver Account, Advantage Fixed Term Deposit account and GoalSaver Account.'

In [106]:
def summarize(context):
    
    vertex_llm_text = VertexAI(model_name="text-bison@001")
    vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001") 
    
       
    prompt_template = """Summarize the information about the three accounts in few lines \n\n
                        Context: \n {context}?\n                       
                        Answer:
                      """

    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context"]
    )
    stuff_chain = load_qa_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)
    stuff_answer = stuff_chain(
    {"input_documents": pages}, return_only_outputs=True
    )
    return stuff_answer["output_text"]

In [107]:
summarize(context)

'The SuperSaver Account is a savings account with a fixed interest rate for 12 months.\nThe Advantage Fixed Term Deposit account is a fixed term deposit account with a fixed interest rate for a term of 6 months, 12 months or 24 months.\nThe GoalSaver Account is a variable rate savings account with no minimum monthly deposit.'